In [2]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder, OneHotEncoder, KBinsDiscretizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV

import lightgbm as lgb
import torch
import torch.nn as nn

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_squared_log_error, mean_absolute_percentage_error, median_absolute_error

In [3]:
target_col = 'close'

In [20]:
# Data created with data_stocks.ipynb
data_raw = pd.read_csv('data/2024-04-20.csv', index_col=False)

C:\Users\18327\AppData\Local\Temp\ipykernel_9020\1909608451.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  data_raw = pd.read_csv('data/2024-04-20.csv', index_col=False)


In [21]:
data_raw 

,datetime,open,high,low,close,volume,ticker,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,2024-04-19,126.05000,127.46000,125.74,127.27,1869200,EA,EA,Electronic Arts,Communication Services,Interactive Home Entertainment,"Redwood City, California",2002-07-22,712515,1982
1,2024-04-18,127.09000,127.39000,125.11,125.63,1389500,EA,EA,Electronic Arts,Communication Services,Interactive Home Entertainment,"Redwood City, California",2002-07-22,712515,1982
2,2024-04-17,126.87000,127.53000,126.29,126.31,1699200,EA,EA,Electronic Arts,Communication Services,Interactive Home Entertainment,"Redwood City, California",2002-07-22,712515,1982
3,2024-04-16,127.72000,128.77000,126.36,126.55,1899900,EA,EA,Electronic Arts,Communication Services,Interactive Home Entertainment,"Redwood City, California",2002-07-22,712515,1982
4,2024-04-15,128.21001,128.39999,126.55,127.05,1419700,EA,EA,Electronic Arts,Communication Services,Interactive Home Entertainment,"Redwood City, California",2002-07-22,712515,1982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472992,1997-05-30,21.12000,22.12000,21.12,22.00,220400,ARE,ARE,Alexandria Real Estate Equities,Real Estate,Office REITs,"Pasadena, California",2017-03-20,1035443,1994
1472993,1997-05-29,21.00000,21.38000,21.00,21.38,296800,ARE,ARE,Alexandria Real Estate Equities,Real Estate,Office REITs,"Pasadena, California",2017-03-20,1035443,1994
1472994,1997-05-28,20.62000,21.38000,20.62,21.00,1862700,ARE,ARE,Alexandria Real Estate Equities,Real Estate,Office REITs,"Pasadena, California",2017-03-20,1035443,1994
1472995,1970-01-01,152.53000,152.53000,152.50,152.50,164699,ARE,ARE,Alexandria Real Estate Equities,Real Estate,Office REITs,"Pasadena, California",2017-03-20,1035443,1994


In [16]:
# Create Previous n Days Closing Price
data = data_raw.copy()

# Convert 'date' column to datetime
data['datetime'] = pd.to_datetime(data['datetime'])

# Sort dataframe by date (if not already sorted)
data = data.sort_values(['ticker', 'datetime'])

# Number of previous days to consider
ticker_days = data.groupby('ticker')['ticker'].value_counts().reset_index()
max_num_days = ticker_days['count'].max()

# TODO: Running out of memory with this.
# Iterate through previous days and create columns for each day
# shifted_columns = [data['close'].shift(i) for i in range(1, max_num_days + 1)]
# data = pd.concat([data] + shifted_columns, axis=1)

MemoryError: Unable to allocate 11.2 MiB for an array with shape (1472997,) and data type float64

In [ ]:
X = data.copy()[target_col]
y = data.copy().drop(target_col, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify='GICS Sector')